## Export Dataset from GEE

In [ ]:
!pip install geemap

import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Qi9yqW4kduII0sFNn9HZvuIkpvAQWHWnsRZMDtP6N2M&tc=GhTRW6x-rWhyDneYObxvMWSK2eSiQsib__lUQKSOn9g&cc=ou76E1dipiS576h30zPhIMeeyp-6xBfdIYaIwEjzIWQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BWwNkUgnjCU6ptHBraVLlt52IzRNacbKpzyhqn9RUJmcpFFBB5BjpI

Successfully saved authorization token.


In [ ]:
Map = geemap.Map()

In [ ]:
NDVI_BAND = "NDVI"
LEAF_BANDS = ["Fpar", "Lai"]
SOIL_MOI_BAND = "SoilMoi00_10cm_tavg"
ERA5_BANDS = ["total_precipitation", "u_component_of_wind_10m", "v_component_of_wind_10m"]
RESPONSE = "T21"
BANDS = [NDVI_BAND] + LEAF_BANDS + [SOIL_MOI_BAND] + ERA5_BANDS

In [ ]:
KERNEL_SIZE = 256

In [ ]:
ndvi = ee.ImageCollection('MODIS/MOD09GA_006_NDVI').filterDate('2019-07-01', '2019-07-14').select(NDVI_BAND).median()
leaf = ee.ImageCollection('MODIS/061/MCD15A3H').filterDate('2019-07-01', '2019-07-14').select(LEAF_BANDS).median()
soil = ee.ImageCollection('NASA/FLDAS/NOAH01/C/GL/M/V001').filterDate('2019-07-01', '2019-07-14').select(SOIL_MOI_BAND).median()
era5 = ee.ImageCollection('ECMWF/ERA5/MONTHLY').filterDate('2019-07-01', '2019-07-14').select(ERA5_BANDS).median()
firms = ee.ImageCollection('FIRMS').filterDate('2019-07-15', '2019-07-28').select(RESPONSE).max().unmask(0).divide(510)

In [ ]:
has_fire = firms.ceil().byte().rename('HAS_FIRE')

In [ ]:
featureStack = ee.Image.cat([ndvi, leaf, soil, era5, firms])

In [ ]:
list1d = ee.List.repeat(1, KERNEL_SIZE)
list2d = ee.List.repeat(list1d, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, list2d)

featureStackArrays = ee.Image.cat([featureStack.neighborhoodToArray(kernel), has_fire])

In [ ]:
trainingPolys = ee.FeatureCollection('projects/google/DemoTrainingGeometries')
evalPolys = ee.FeatureCollection('projects/google/DemoEvalGeometries')

trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())

In [ ]:
for g in range(trainingPolys.size().getInfo()):
  geoSample = ee.FeatureCollection([])
  for i in range(10):
      samples = featureStackArrays.stratifiedSample(
          10,
          classBand = "HAS_FIRE",
          region = ee.Feature(trainingPolysList.get(g)).geometry(),
          scale = 30,
          geometries = False,
          seed = i
      )
      geoSample = geoSample.merge(samples)

  task = ee.batch.Export.table.toDrive(
    collection = geoSample,
    folder = 'GEE',
    description = "export training data - wildf-" + str(g),
    fileNamePrefix = 'wf_train_' + str(g),
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )

  task.start()

In [ ]:
for g in range(evalPolys.size().getInfo()):
  geoSample = ee.FeatureCollection([])
  for i in range(10):
      samples = featureStackArrays.stratifiedSample(
          10,
          classBand = "HAS_FIRE",
          region = ee.Feature(evalPolysList.get(g)).geometry(),
          scale = 30,
          geometries = False,
          seed = i
      )
      geoSample = geoSample.merge(samples)

  task = ee.batch.Export.table.toDrive(
    collection = geoSample,
    folder = 'GEE',
    description = "export val data - wildf-" + str(g),
    fileNamePrefix = 'wf_val_' + str(g),
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )

  task.start()

## Training - TensorFlow with UNet

In [ ]:
# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)

2.12.0


In [ ]:
FEATURES = BANDS + [RESPONSE]
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns:
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns:
    A tuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns:
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

In [ ]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns:
    A tf.data.Dataset of training data.
  """
	glob = '/content/drive/MyDrive/GEE/wf_train_*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(500).batch(16).repeat()
	return dataset

training = get_training_dataset()

print(iter(training.take(1)).next())

(<tf.Tensor: shape=(16, 256, 256, 7), dtype=float32, numpy=
array([[[[ 5.32797158e-01,  4.80000000e+01,  1.30000000e+01, ...,
           4.68245819e-02, -4.96423304e-01,  1.66753232e+00],
         [ 5.32797158e-01,  4.80000000e+01,  1.30000000e+01, ...,
           4.68245819e-02, -4.96423304e-01,  1.66753232e+00],
         [ 5.32797158e-01,  4.80000000e+01,  1.30000000e+01, ...,
           4.68245819e-02, -4.96423304e-01,  1.66753232e+00],
         ...,
         [ 3.00558984e-01,  5.90000000e+01,  1.70000000e+01, ...,
           4.68245819e-02, -4.96423304e-01,  1.66753232e+00],
         [ 3.00558984e-01,  5.90000000e+01,  1.70000000e+01, ...,
           4.68245819e-02, -4.96423304e-01,  1.66753232e+00],
         [ 3.00558984e-01,  5.90000000e+01,  1.70000000e+01, ...,
           4.68245819e-02, -4.96423304e-01,  1.66753232e+00]],

        [[ 5.32797158e-01,  4.80000000e+01,  1.30000000e+01, ...,
           4.68245819e-02, -4.96423304e-01,  1.66753232e+00],
         [ 5.32797158e-01,  

In [ ]:
def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns:
    A tf.data.Dataset of evaluation data.
  """
	glob = '/content/drive/MyDrive/GEE/wf_val_*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()

In [ ]:
OPTIMIZER = 'adam'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

In [ ]:
def conv_block(input_tensor, num_filters):
	encoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = tf.keras.layers.BatchNormalization()(encoder)
	encoder = tf.keras.layers.Activation('relu')(encoder)
	encoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = tf.keras.layers.BatchNormalization()(encoder)
	encoder = tf.keras.layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = tf.keras.layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = tf.keras.layers.BatchNormalization()(decoder)
	decoder = tf.keras.layers.Activation('relu')(decoder)
	decoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = tf.keras.layers.BatchNormalization()(decoder)
	decoder = tf.keras.layers.Activation('relu')(decoder)
	decoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = tf.keras.layers.BatchNormalization()(decoder)
	decoder = tf.keras.layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = tf.keras.layers.Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=tf.keras.optimizers.get(OPTIMIZER),
		loss=tf.keras.losses.get(LOSS),
		metrics=[tf.keras.metrics.get(metric) for metric in METRICS])

	return model

In [ ]:
m = get_model()

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

m.fit(
    x=training,
    epochs=200,
    steps_per_epoch=100,
    validation_data=evaluation,
    validation_steps=50,
    callbacks=[callback])

m.save('/content/drive/MyDrive/GEE/0803wf.keras')

Epoch 1/200
100/100 [==============================] - 68s 457ms/step - loss: 0.0410 - root_mean_squared_error: 0.2026 - val_loss: 0.1312 - val_root_mean_squared_error: 0.3622
Epoch 2/200
100/100 [==============================] - 44s 436ms/step - loss: 0.0340 - root_mean_squared_error: 0.1845 - val_loss: 0.8837 - val_root_mean_squared_error: 0.9401
Epoch 3/200
100/100 [==============================] - 44s 443ms/step - loss: 0.0324 - root_mean_squared_error: 0.1801 - val_loss: 0.6338 - val_root_mean_squared_error: 0.7961
Epoch 4/200
100/100 [==============================] - 42s 416ms/step - loss: 0.0295 - root_mean_squared_error: 0.1717 - val_loss: 0.4620 - val_root_mean_squared_error: 0.6797
Epoch 5/200
100/100 [==============================] - 42s 420ms/step - loss: 0.0293 - root_mean_squared_error: 0.1712 - val_loss: 0.3201 - val_root_mean_squared_error: 0.5657
Epoch 6/200
100/100 [==============================] - 42s 424ms/step - loss: 0.0263 - root_mean_squared_error: 0.1623 -

## Predict with Trained Model

In [ ]:
m = tf.keras.models.load_model('/content/drive/MyDrive/GEE/0803wf.keras')

In [ ]:
def doExport(out_image_base, kernel_buffer, region):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toDrive(
    image = featureStack.select(BANDS),
    description = out_image_base,
    folder = "GEE",
    fileNamePrefix = out_image_base,
    region = region.getInfo()['coordinates'],
    scale = 30,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Drive...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [ ]:
def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  # filesList = !gsutil ls "/content/drive/MyDrive/GEE"

  # Get only the files generated by the image export.
  # exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = ['/content/drive/MyDrive/GEE/wf_malibu_-00000.tfrecord.gz','/content/drive/MyDrive/GEE/wf_malibu_-00001.tfrecord.gz']
  jsonFile = '/content/drive/MyDrive/GEE/wf_malibu_-mixer.json'
  # for f in exportFilesList:
  #   if f.endswith('.tfrecord.gz'):
  #     imageFilesList.append(f)
  #   elif f.endswith('.json'):
  #     jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)

  import json
  # Load the contents of the mixer file to a JSON object.
  # jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  # mixer = json.loads(jsonText.nlstr)
  f = open(jsonFile)
  mixer = json.load(f)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  out_image_file = '/content/drive/MyDrive/GEE/Prediction' + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'T21': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

In [ ]:
# Output assets folder: YOUR FOLDER
user_folder = 'ee-yzyly1992' # INSERT YOUR FOLDER HERE.

# Base file name to use for TFRecord files and assets.
bj_image_base = 'wf_malibu_'
# Half this will extend on the sides of each patch.
bj_kernel_buffer = [128, 128]
# Malibu
bj_region = ee.Geometry.Polygon(
        [[[-118.42352813969785,33.997424855281345],
          [-118.42352813969785,34.25094039380779],
          [-119.15274566899473,34.25094039380779],
          [-119.15274566899473,33.997424855281345]]], None, False)

In [ ]:
# Run the export.
doExport(bj_image_base, bj_kernel_buffer, bj_region)

Running image export to Drive...
Image export completed.


In [ ]:
# Run the prediction.
doPrediction(bj_image_base, user_folder, bj_kernel_buffer, bj_region)

Looking for TFRecord files...
['/content/drive/MyDrive/GEE/wf_malibu_-00000.tfrecord.gz',
 '/content/drive/MyDrive/GEE/wf_malibu_-00001.tfrecord.gz']
/content/drive/MyDrive/GEE/wf_malibu_-mixer.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 10,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            -119.1529884432554,
                                            0.0,
                                            -0.00026949458523585647,
                                            34.251683805136416]},
                'crs': 'EPSG:4326'},
 'totalPatches': 30}
Running predictions...
30/30 [==============================] - 11s 31ms/step
Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Writing patch 4...
Writing patch 5...
Writing patch 6...
Writing patch 7...
Writing patch 8...
Writing patch 9...
Writing patch 10...
Writing patch

In [ ]:
out_image = ee.Image('projects/ee-yzyly1992/assets/wf-malibu')

In [ ]:
Map.addLayer(out_image,
             {'min': 0, 'max': 1, 'palette':['green', 'orange', 'red']},
             'wildfire',False)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…